In [2]:
import matplotlib.pyplot as plt

plt.style.use("seaborn")
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score, log_loss
import torch
from datetime import date
from nnet import ReadDataset, Net, ResNet


def average_NN(
    data,
    weights=[
        "output/weights70.pt",
        "output/weights80.pt",
        "output/90.pt",
        "output/weights_final.pt",
    ],
):
    predicciones = []
    for w in weights:
        NNet = ResNet(data.shape[0]).to(device)
        NNet.load_state_dict(torch.load(w))
        predicciones.append(NNet.forward(X_train).detach().numpy())
    return np.mean(predicciones, axis=1)


def evaluate_auc(model, data, label, ensemble=False):
    if ensemble:
        pred = average_NN(data)
    else:
        pred = model(data.float()).detach().numpy()
    return roc_auc_score(label.detach().numpy(), pred)


def evaluate_log_loss(model, data, label, ensemble=False):
    if ensemble:
        pred = average_NN(data)
    else:
        pred = model(data.float()).detach().numpy()
    return log_loss(label.detach().numpy(), pred)


/opt/anaconda3/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


In [5]:
# Train
dataset = ReadDataset("../data/train_split.csv")
# Transform to tensor
X_train = torch.tensor(dataset.X.values).to(device).float()
y_train = torch.tensor(dataset.y.values).to(device).float()


FileNotFoundError: [Errno 2] No such file or directory: 'output/scaler.save'

In [ ]:


# Predictions
print("Results in train AUC: ", evaluate_auc(nnet, X_train, y_train))


print("Results in train AUC: ", evaluate_auc(nnet, X_train, y_train,ense))
